Here, we compare the components of our identity when we fit with weighted least squares versus when we fit with ordinary least squares.

# Imports:

In [23]:
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import pickle
import statsmodels.api as sm
import random
import scipy.stats as stats

# Hyperparameters:

In [29]:
pop_size = 100 * 1_000
sample_size = 30
beta = 1
num_iters = 100_000

In [30]:
## for rng generator:
pop_number = 1

# Run:

In [31]:
rand_generator = np.random.default_rng(seed=333 * pop_number)

In [32]:
fitted_betas = []
fitted_FI = []
fitted_correlations = []
gstar_variances = []

Create the simulated data (x, w, y):

In [33]:
assert pop_size % 100 == 0
base_x = [i for i in range(-50, 0)] + [i for i in range(1, 51)]
x = pd.Series(base_x * (pop_size // len(base_x)))

w = x.abs()
data = pd.concat([x, w], axis=1)
data.columns = ['x', 'w']
data['x_weighted'] = data['x'] / data['w']

rand_generator = np.random.default_rng(seed=333 * pop_number)

data['y'] = data['x'] * beta + rand_generator.normal(size=pop_size) * data['w']
data['y_weighted'] = data['y'] / data['w']

ols_fitted_beta = sm.OLS(data['y'], data['x']).fit().params.iloc[0]
wls_fitted_beta = sm.OLS(data['y_weighted'], data['x_weighted']).fit().params.iloc[0]
data['g_star'] = data['x'] * (data['y'] - data['x'] * ols_fitted_beta)
data['g_star_weighted'] = data['x_weighted'] * (data['y_weighted'] - data['x_weighted'] * wls_fitted_beta)

Run random samples:

In [34]:
for _ in trange(num_iters):
    srs_sample_indices = pd.Series(random.sample(range(pop_size), sample_size))
    srs_sample_indices = srs_sample_indices.sort_values().reset_index(drop=True)

    data['r_srs'] = 0
    data.loc[srs_sample_indices, 'r_srs'] = 1
    srs_sample = data[data['r_srs'] == 1]

    srs_fitted_beta = sm.OLS(srs_sample['y'], srs_sample['x']).fit().params.iloc[0]
    other_fitted_beta = sm.OLS(srs_sample['y_weighted'], srs_sample['x_weighted']).fit().params.iloc[0]

    fitted_betas.append(pd.Series([ols_fitted_beta, wls_fitted_beta, \
                           srs_fitted_beta, other_fitted_beta]))

    fitted_FI.append(pd.Series([np.mean(srs_sample['x'] ** 2), np.mean(srs_sample['x_weighted'] ** 2)]))

    fitted_correlations.append(data[['g_star', 'g_star_weighted', 'r_srs']].corr().iloc[0:2, 2])

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [06:34<00:00, 253.72it/s]


In [35]:
gstar_variances = data[['g_star', 'g_star_weighted']].var()

  3%|██                                                                       | 2797/100000 [00:22<13:22, 121.13it/s]


  4%|██▊                                                                      | 3786/100000 [00:29<12:22, 129.51it/s]


  5%|███▍                                                                     | 4776/100000 [00:36<11:24, 139.11it/s]


  6%|████▏                                                                    | 5794/100000 [00:44<10:57, 143.39it/s]


  7%|████▉                                                                    | 6817/100000 [00:51<11:31, 134.83it/s]


...

 97%|█████████████████████████████████████████████████████████████████████▊  | 96953/100000 [13:29<00:26, 113.38it/s]


 98%|██████████████████████████████████████████████████████████████████████▍ | 97792/100000 [13:37<00:19, 115.05it/s]


 99%|███████████████████████████████████████████████████████████████████████ | 98693/100000 [13:44<00:10, 123.59it/s]


100%|███████████████████████████████████████████████████████████████████████▋| 99575/100000 [13:52<00:03, 122.03it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 1/1 [13:55<00:00, 835.70s/it]


In [36]:
gstar_variances

g_star             1.321442e+06
g_star_weighted    1.009252e+00
dtype: float64

##### post-process data:

In [37]:
fitted_betas = pd.concat(fitted_betas, axis=1).T.reset_index(drop=True)
fitted_FI = pd.concat(fitted_FI, axis=1).T.reset_index(drop=True)
fitted_correlations = pd.concat(fitted_correlations, axis=1).T.reset_index(drop=True)

In [38]:
display(fitted_FI.head())
display(fitted_betas.head())
display(fitted_correlations.head())

,0,1
0,602.500000,1.0
1,1201.633333,1.0
2,911.866667,1.0
3,848.366667,1.0
4,812.566667,1.0


,0,1,2,3
0,0.999535,1.002907,0.860980,1.134152
1,0.999535,1.002907,0.814275,0.974034
2,0.999535,1.002907,1.132230,1.089826
3,0.999535,1.002907,0.873447,0.888596
4,0.999535,1.002907,1.119822,1.078431


,g_star,g_star_weighted
0,-0.001258,0.002263
1,-0.003355,-0.000498
2,0.001823,0.001499
3,-0.001612,-0.001971
4,0.001473,0.001302


In [39]:
fitted_FI.columns = ['srs', 'wgted']
fitted_betas.columns = ['ols', 'wls', 'srs', 'wgted']
fitted_correlations.columns = ['srs', 'wgted']

In [40]:
(fitted_correlations['srs'] ** 2).corr(fitted_FI['srs'] ** 2)

np.float64(0.15926991715619543)

# explore results:

##### compute JEEE:

In [41]:
jeee_ols = np.mean(data['x'] ** 2)
jeee_wls = np.mean(data['x_weighted'] ** 2)

In [42]:
jeee_ols

np.float64(858.5)

In [43]:
jeee_wls

np.float64(1.0)

## Compare MSEs of ols vs wls:

In [44]:
fitted_betas['srs_diff_sq'] = (fitted_betas['ols'] - fitted_betas['srs']) ** 2
fitted_betas['wgted_diff_sq'] = (fitted_betas['wls'] - fitted_betas['wgted']) ** 2

In [45]:
fitted_betas.head()

,ols,wls,srs,wgted,srs_diff_sq,wgted_diff_sq
0,0.999535,1.002907,0.860980,1.134152,0.019197,0.017225
1,0.999535,1.002907,0.814275,0.974034,0.034321,0.000834
2,0.999535,1.002907,1.132230,1.089826,0.017608,0.007555
3,0.999535,1.002907,0.873447,0.888596,0.015898,0.013067
4,0.999535,1.002907,1.119822,1.078431,0.014469,0.005704


In [46]:
averaged_fitted_betas = fitted_betas.mean()

In [83]:
fitted_betas

,ols,wls,srs,wgted,srs_diff_sq,wgted_diff_sq
0,0.999535,1.002907,0.860980,1.134152,0.019197,0.017225
1,0.999535,1.002907,0.814275,0.974034,0.034321,0.000834
2,0.999535,1.002907,1.132230,1.089826,0.017608,0.007555
3,0.999535,1.002907,0.873447,0.888596,0.015898,0.013067
4,0.999535,1.002907,1.119822,1.078431,0.014469,0.005704
...,...,...,...,...,...,...
99995,0.999535,1.002907,1.009602,1.001798,0.000101,0.000001
99996,0.999535,1.002907,1.153654,1.088049,0.023753,0.007249
99997,0.999535,1.002907,1.567970,1.290063,0.323119,0.082459
99998,0.999535,1.002907,1.381708,1.131474,0.146057,0.016529


In [49]:
(averaged_fitted_betas['wgted_diff_sq'] / averaged_fitted_betas['srs_diff_sq'])

np.float64(0.5618998949856291)

In [50]:
srs_mean, srs_std = (
    fitted_betas['srs_diff_sq'].mean(), 
    fitted_betas['srs_diff_sq'].std() / np.sqrt(num_iters)
)
biased_mean, biased_std = (
    fitted_betas['wgted_diff_sq'].mean(), 
    fitted_betas['wgted_diff_sq'].std() / np.sqrt(num_iters)
)

In [51]:
srs_mean, srs_std

(np.float64(0.060081161572709534), np.float64(0.00027137418700732))

In [52]:
biased_mean, biased_std

(np.float64(0.0337595983783201), np.float64(0.00015156182942124087))

In [53]:
biased_mean / srs_mean

np.float64(0.5618998949856291)

In [54]:
# Calculate the 97.5th quantile, to compute rough confidence intervals:
quantile_77 = stats.norm.ppf(0.9875)
print(1 - 0.025 / 2)
print(quantile_77)

0.9875
2.241402727604947


In [55]:
srs_lb, srs_ub = srs_mean - srs_std * quantile_77, srs_mean + srs_std * quantile_77
biased_lb, biased_ub = biased_mean - biased_std * quantile_77, biased_mean + biased_std * quantile_77

In [56]:
biased_lb / srs_ub, biased_ub / srs_lb

(np.float64(0.5506707273122335), np.float64(0.5733587551819357))

In [57]:
biased_lb / srs_ub, biased_ub / srs_lb

(np.float64(0.5506707273122335), np.float64(0.5733587551819357))

### look at relative sample elasticity:

In [59]:
(fitted_FI['wgted'] / jeee_wls).mean()

np.float64(1.0)

In [60]:
(fitted_FI['srs'] / jeee_ols).mean()

np.float64(0.9999565369831102)

In [99]:
(jeee_wls / fitted_FI['wgted']).mean()

np.float64(1.0)

In [97]:
(jeee_ols / fitted_FI['srs']).mean()

np.float64(1.0280296380471632)

In [61]:
(fitted_correlations['srs'] ** 2).corr(fitted_FI['srs'] ** 2)

np.float64(0.15926991715619543)

# Compare the elasticities and correlations:

In [85]:
fitted_sample_elasticity = fitted_FI.copy()

fitted_sample_elasticity['srs'] /= jeee_ols
fitted_sample_elasticity['wgted'] /= jeee_wls

In [86]:
fitted_sample_elasticity.describe()

,srs,wgted
count,100000.000000,100000.0
mean,0.999957,1.0
std,0.161920,0.0
min,0.302931,1.0
25%,0.888798,1.0
50%,0.996622,1.0
75%,1.108018,1.0
max,1.672297,1.0


In [87]:
fitted_correlations.describe()

,srs,wgted
count,100000.000000,100000.000000
mean,-0.000012,-0.000013
std,0.003169,0.003168
min,-0.018546,-0.012961
25%,-0.002109,-0.002156
50%,-0.000005,-0.000015
75%,0.002082,0.002126
max,0.016789,0.014246


In [91]:
(fitted_correlations['srs'] ** 2).corr(1/(fitted_sample_elasticity['srs'] ** 2))

np.float64(-0.14588905149089382)

In [96]:
(fitted_correlations['srs']).corr(1/(fitted_sample_elasticity['srs']))

np.float64(0.003812039294839382)

In [92]:
(fitted_correlations['wgted'] ** 2).corr(1/(fitted_sample_elasticity['wgted'] ** 2))

np.float64(nan)

In [93]:
np.mean((fitted_correlations['srs'] ** 2)/(fitted_sample_elasticity['srs'] ** 2))

np.float64(1.0056031240392565e-05)

In [94]:
np.mean((fitted_correlations['wgted'] ** 2)/(fitted_sample_elasticity['wgted'] ** 2))

np.float64(1.0038150085301142e-05)

In [95]:
np.std((fitted_correlations['srs'] ** 2)/(fitted_sample_elasticity['srs'] ** 2))

np.float64(1.4363314083236781e-05)

In [68]:
fitted_correlations['srs'] ** 2 * fitted_FI['srs']

0        1.582575e-06
1        1.125413e-05
2        3.324922e-06
3        2.598502e-06
4        2.169522e-06
             ...     
99995    1.272054e-08
99996    3.632303e-06
99997    4.171331e-05
99998    1.492502e-05
99999    1.135439e-05
Name: srs, Length: 100000, dtype: float64

In [82]:
fitted_sample_elasticity

,srs,wgted
0,0.000456,0.990833
1,0.000909,0.990833
2,0.000690,0.990833
3,0.000642,0.990833
4,0.000615,0.990833
...,...,...
99995,0.000563,0.990833
99996,0.000621,0.990833
99997,0.000571,0.990833
99998,0.000508,0.990833


In [66]:
fitted_FI['srs'] / gstar_variances['gstar']

,srs,wgted
0,602.500000,1.0
1,1201.633333,1.0
2,911.866667,1.0
3,848.366667,1.0
4,812.566667,1.0
...,...,...
99995,743.400000,1.0
99996,820.600000,1.0
99997,753.966667,1.0
99998,670.800000,1.0


In [62]:
gstar_variances

g_star             1.321442e+06
g_star_weighted    1.009252e+00
dtype: float64

In [63]:
gstar_variances['g_star'] / (jeee_ols ** 2)

np.float64(1.7929476616965772)

In [64]:
variances_and_fi = fitted_FI.join(gstar_variances, rsuffix = '_r', lsuffix = '_l')

ValueError: Other Series must have a name

In [45]:
fitted_corr_0th = fitted_correlations[fitted_correlations['sim_group'] == 0].copy()
fitted_fi_0th = fitted_FI[fitted_FI['sim_group'] == 0].copy()

In [51]:
fitted_fi_0th_sq = fitted_fi_0th ** 2

In [52]:
gstar_variances_0th = gstar_variances[gstar_variances['sim_group'] == 0].copy()

In [56]:
variances_and_fi = fitted_fi_0th_sq.join(gstar_variances_0th, rsuffix = '_r', lsuffix = '_l')

In [60]:
variances_and_fi[['g_star', 'g_star_weighted']] = variances_and_fi[['g_star', 'g_star_weighted']].fillna(method = 'ffill')

In [61]:
(variances_and_fi['g_star'] / variances_and_fi['srs']).mean(), \
        (variances_and_fi['g_star'] / variances_and_fi['srs']).std() / np.sqrt(num_iters) * 2

(1.9505843964348692, 0.004549235973205695)

In [62]:
variances_and_fi

,srs,biased,sim_group_l,g_star,g_star_weighted,sim_group_r
0,333313.777778,1.0,0,1.321442e+06,1.009252,0.0
1,411222.937778,1.0,0,1.321442e+06,1.009252,NaN
2,721990.090000,1.0,0,1.321442e+06,1.009252,NaN
3,739886.694444,1.0,0,1.321442e+06,1.009252,NaN
4,437097.284444,1.0,0,1.321442e+06,1.009252,NaN
...,...,...,...,...,...,...
99995,729999.360000,1.0,0,1.321442e+06,1.009252,NaN
99996,500414.760000,1.0,0,1.321442e+06,1.009252,NaN
99997,892080.250000,1.0,0,1.321442e+06,1.009252,NaN
99998,825130.001111,1.0,0,1.321442e+06,1.009252,NaN


In [29]:
squared_fitted_correlations = (fitted_correlations ** 2)
averaged_squared_fitted_correlations = squared_fitted_correlations.groupby(by='sim_group').mean()

squared_fitted_FI = fitted_FI ** 2
averaged_squared_fitted_FI = squared_fitted_FI.groupby(by='sim_group').mean()

In [30]:
(averaged_squared_fitted_FI['srs'] / averaged_squared_fitted_FI['biased']).describe()

count      1000.000000
mean     755949.432124
std        7653.879194
min      727891.466366
25%      750920.207935
50%      756063.900359
75%      761279.169486
max      778325.327227
dtype: float64

In [31]:
0.460773 * 1.529332

0.7046748936359999

In [32]:
(averaged_squared_fitted_correlations['srs'] / averaged_squared_fitted_correlations['biased']).describe()

count    1000.000000
mean        1.001207
std         0.045181
min         0.857312
25%         0.970707
50%         1.000381
75%         1.031993
max         1.164470
dtype: float64

In [34]:
fitted_betas['srs_diff_sq'] = (fitted_betas['ols'] - fitted_betas['srs']) ** 2
fitted_betas['biased_diff_sq'] = (fitted_betas['wls'] - fitted_betas['biased']) ** 2

In [70]:
fitted_betas.head()

,ols,wls,srs,biased,sim_group,srs_diff_sq,biased_diff_sq
0,0.999535,1.002907,1.056136,1.161585,0,0.003204,0.025179
1,0.999535,1.002907,1.065845,0.970815,0,0.004397,0.001030
2,0.999535,1.002907,1.456815,1.160359,0,0.209105,0.024791
3,0.999535,1.002907,0.821681,0.881015,0,0.031632,0.014858
4,0.999535,1.002907,1.386928,0.974017,0,0.150074,0.000835


In [41]:
averaged_fitted_betas = fitted_betas.groupby(by='sim_group').mean()

In [42]:
(averaged_fitted_betas['biased_diff_sq'] / averaged_fitted_betas['srs_diff_sq']).describe()

count    1000.000000
mean        0.559495
std         0.025386
min         0.480251
25%         0.542454
50%         0.557838
75%         0.576719
max         0.650302
dtype: float64

In [49]:
s0_betas = fitted_betas[fitted_betas['sim_group'] == 0].copy()

In [56]:
srs_mean, srs_std = s0_betas['srs_diff_sq'].mean(), s0_betas['srs_diff_sq'].std() / np.sqrt(num_iters)
biased_mean, biased_std = s0_betas['biased_diff_sq'].mean(), s0_betas['biased_diff_sq'].std() / np.sqrt(num_iters)

In [57]:
srs_mean, srs_std

(0.05937972209881229, 0.0027750321937486315)

In [58]:
biased_mean, biased_std

(0.034113937464896694, 0.0015701325935451342)

In [69]:
biased_mean / srs_mean

0.574504835306042

In [65]:
import scipy.stats as stats

# Calculate the 97.5th quantile
quantile_77 = stats.norm.ppf(0.9875)
print(1 - 0.025 / 2)
print(quantile_77)

0.9875
2.241402727604947


In [66]:
srs_lb, srs_ub = srs_mean - srs_std * quantile_77, srs_mean + srs_std * quantile_77
biased_lb, biased_ub = biased_mean - biased_std * quantile_77, biased_mean + biased_std * quantile_77

In [68]:
biased_lb / srs_ub, biased_ub / srs_lb

(0.46638390313773953, 0.7079271765754301)

In [38]:
srs_squared_fi_and_corr = squared_fitted_FI[['srs', 'sim_group']].join( \
                                        squared_fitted_correlations['srs'], rsuffix='_corr', lsuffix='_fi')

biased_squared_fi_and_corr = squared_fitted_FI[['biased', 'sim_group']].join( \
                                        squared_fitted_correlations['biased'], rsuffix='_corr', lsuffix='_fi')

In [47]:
biased_fi_corr_corr = biased_squared_fi_and_corr.groupby(by='sim_group')[['biased_fi', 'biased_corr']].corr().reset_index()

In [48]:
biased_fi_corr_corr[biased_fi_corr_corr['level_1'] == 'biased_corr']['biased_fi'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: biased_fi, dtype: float64

In [49]:
srs_fi_corr_corr = srs_squared_fi_and_corr.groupby(by='sim_group')[['srs_fi', 'srs_corr']].corr().reset_index()

In [50]:
srs_fi_corr_corr[srs_fi_corr_corr['level_1'] == 'srs_corr']['srs_fi'].describe()

count    1.000000
mean     0.149926
std           NaN
min      0.149926
25%      0.149926
50%      0.149926
75%      0.149926
max      0.149926
Name: srs_fi, dtype: float64